# Volume Rendering with yt (+ extensions)

## yt-napari 

napari:
* interactive
* mostly max intensity projections

yt-napari:
* wrappers to convert yt fields to fix-resolution arrays to load as an image array



```
pip install napari[pyQT5] yt-napari 
```

In [ ]:
import napari
from yt_napari.viewer import Scene
import yt 


ds = yt.load("IsolatedGalaxy/galaxy0030/galaxy0030")
viewer = napari.Viewer()

yt_scene = Scene()

left_edge = ds.domain_center - ds.arr([10, 10, 10], 'kpc')
right_edge = ds.domain_center + ds.arr([10, 10, 10], 'kpc')
yt_scene.add_region(viewer,
                    ds,
                    ("enzo", "Density"),
                    left_edge = left_edge,
                    right_edge = right_edge,
                    resolution=(600, 600, 600),
                    colormap="magma",
                    contrast_limits=(3.4, 6.4))

yt_scene.add_region(viewer,
                    ds,
                    ("enzo", "Temperature"),
                    left_edge = left_edge,
                    right_edge = right_edge,
                    resolution=(300, 300, 300),
                    colormap="yellow",
                    contrast_limits=(4., 6.),
                    opacity=0.6)

# yt-idv 

The good:

* OpenGL, interactive and headless modes
* direct rendering AMR and octree grids
* max intensity and integrative projections (and custom transfer functions...)
* and now direct volume rendering of data in spherical coordinates!

The bad:

* need to fit index and field in memory
* can be a little tricky to install

**if on ubuntu**: wayland graphics (the default with ubuntu>=22.04) do not work (cause of one of yt_idv's dependencies...): select "ubuntu on xorg" from login screen https://help.symless.com/hc/en-us/articles/35748582406545-Switch-back-to-Xorg-on-Ubuntu 

**if on mac**: interactive works well, headless rendering is harder (install osmesa vs conda, but sloooooow). 

**windows**: &#128556;

Example in this directory:

```shell
pip install yt_idv 
python volume_rendering_03_yt_idv_spherical.py --domain partial
```

# yt

https://yt-project.org/doc/visualizing/volume_rendering.html


![](figures/yt_vr_scene.png)


## Sources 

![](figures/render_source.png)

(from https://chrishavlin.github.io/post/yt-vr-w-streamlines/)

* Volume sources: ray tracing and a transfer function to sample fields
* Opaque sources: points, lines, etc that are ... opaque


## Volume Sources


### Transfer functions

In [ ]:
import yt

ds = yt.load_sample("Enzo_64")

sc = yt.create_scene(ds, lens_type="perspective") # creates a VolumeSource for you

source = sc[0]  

# Set transfer function properties
source.tfh.set_bounds((3e-31, 5e-27))
source.tfh.set_log(True)
source.tfh.grey_opacity = False

source.tfh.plot(profile_field=("gas", "density"))

In [ ]:
sc.show()  

First: try adjusting the contrast!

In [ ]:
for sig_clip in range(20, -1, -5):
    sc.show(sigma_clip=sig_clip)

In [ ]:
sc.show(sigma_clip=6)

In [ ]:
sc.save_annotated('figures/Enzo64_annotated.png', 
                  sigma_clip=8.,
                  render=False)

![](figures/Enzo64_annotated.png)

# Customizing Transfer functions



In [ ]:
sc = yt.create_scene(ds, lens_type="perspective")

source = sc[0]

source.set_field(("gas", "density"))
source.set_log(True)

bounds = (3e-31, 5e-27)

# Since this rendering is done in log space, the transfer function needs
# to be specified in log space.
tf = yt.ColorTransferFunction(np.log10(bounds))

tf.add_gaussian(np.log10(1e-29), width=0.005, height=[0., 1.0, 0, 1.0])

source.tfh.tf = tf
source.tfh.bounds = bounds

source.tfh.plot(profile_field=("gas", "density"))


In [ ]:
tf.add_gaussian(np.log10(1e-28), width=0.01, height=[0.753, 0.0, 0.933, 1.0])
source.tfh.plot(profile_field=("gas", "density"))

In [ ]:
sc.show(sigma_clip=5)

tf.sample_colormap to add gaussian sampling from a colormap

## Cartesian only, but interpolation can get you pretty far... 


![](figures/havlin_ec20_vr.png)

(from https://github.com/earthcube2020/ec20_havlin_etal)

   
1. interpolate from 3D geodetic coordinates (latitude, longitude, depth) to cartesian grid (inverse-distance weighting with kdtree), load with `yt.load_uniform_grid`, create a scene, add `VolumeSource`
2. domain and shapefile annotations:  tectonic boundaries (Coffin et al., 1998), sites of volcanism of the last 10,000 years (Simkin and Siebert, 1994) and US political boundaries (Natural Earth, https://www.naturalearthdata.com/). defined as (lat, lon) on earth's surface, project to 3d positions and use `LineSource` and `PointSource` objects with `yt.add_source`

### now with `yt_xarray`

Embedded interpolations on demand within yt grids (with `load_amr_grids`):

![](figures/wus_composite_vr.png)

(from https://chrishavlin.github.io/NASASoftwareWorkshop2024/yt-xr-dev-02-transformations.html )


```
pip install yt_xarary scipy xarray netCDF4 
```

```
wget https://yt2025data.hub.yt/geo/wUS-SH-2010_percent.nc
```


In [ ]:
import xarray as xr
import yt_xarray
import yt
from yt_xarray import transformations as tf
from yt_xarray.utilities.logging import ytxr_log
import numpy as np 
import cartopy
from yt.visualization.volume_rendering.render_source import LineSource

import shapely
import cartopy.feature as cfeature

import matplotlib.pyplot as plt 
import unyt

ds = yt_xarray.open_dataset("IRIS/wUS-SH-2010_percent.nc")
grid_resolution = (32, 32, 32)
gc = tf.GeocentricCartesian(radial_type='depth', r_o=6371., use_neg_lons=True)
ds_yt = tf.build_interpolated_cartesian_ds(
    ds,
    gc,
    fields = 'dvs' ,   
    grid_resolution = grid_resolution, 
    refine_grid=True,    
    refine_max_iters=2000,
    refine_min_grid_size=4,
    refine_by=4,
    interp_method='interpolate',
)

# add a nice field... volume rendering with observation data is hard...
# - fill in nans
# - full dvs is +/-, look at just -, make it + 
def _slow_vels(field, data):    
    dvs = data['dvs'].d.copy()
    dvs[np.isnan(dvs)] = 0.0
    dvs[dvs>0] = 0.0
    return unyt.unyt_array(np.abs(dvs),"")

ds_yt.add_field(
    name=("stream", "slow_dvs"),
    function=_slow_vels,
    sampling_type="local",
    units="",    
)

reg = ds_yt.region(ds_yt.domain_center, 
                   ds_yt.domain_left_edge, 
                   ds_yt.domain_right_edge)
sc = yt.create_scene(reg, field=('stream', 'slow_dvs'))
cam = sc.add_camera(ds_yt)

# transfer function 
source = sc[0]
source.tfh.set_bounds((0.1, 8))
source.tfh.set_log(True)

# add state outlines
# sc.add_source(lsrc)

# adjust camera
cam.zoom(2)
cam.yaw(100*np.pi/180)
cam.roll(220*np.pi/180)
cam.rotate(30*np.pi/180)
cam.set_resolution((300,300))

sc.show(sigma_clip=5.)

In [ ]:
sc.save_annotated('figures/wus_live_annotated.png', 
                  sigma_clip=5.,
                  render=False)

![](figures/wus_live_annotated.png)